## Precarga de librerias y funciones

In [105]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Note: you may need to restart the kernel to use updated packages.


In [106]:
%run "../funciones.ipynb"

#### Excel

In [107]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

dataset = dataset_10_meses_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [108]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [109]:
df_train['foto_mes'].unique()

array([202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106,
       202107, 202108], dtype=int64)

In [110]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106


In [111]:
data = df_train

In [112]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [113]:
data['clase_peso'].isna().sum()

0

In [114]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [115]:
data['foto_mes'].unique()

array([202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106,
       202107, 202108], dtype=int64)

In [116]:
# Especificar mes de train y test
meses = [202011, 202012, 202101, 202102, 202103, 202104]


df_train = data[data['foto_mes'].isin(meses)]
df_test = data[data['foto_mes']==mes_test]


In [117]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [118]:
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

In [119]:
df_train['foto_mes'].unique()

array([202011, 202012, 202101, 202102, 202103, 202104], dtype=int64)

## Predicciones

In [120]:
best_params = {'num_leaves': 3545,
 'learning_rate': 0.07959540527561224,
 'min_data_in_leaf': 933,
 'feature_fraction': 0.6829932939284065,
 'bagging_fraction': 0.6251908881121132}

In [121]:
numeros_random = np.random.randint(0, 100000, 20)

In [122]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)



df_voting = pd.DataFrame()

for semilla in numeros_random: 
        
    best_iter = 5
    df_modelos = pd.DataFrame
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'seed': semilla,
    'verbose': 0
    }
        
        
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    y_pred = model.predict(X_test)
    
    df_voting[f'pred_{semilla}'] = y_pred
    
df_voting['pred_mean'] = df_voting.mean(axis=1)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [123]:
y_pred = df_voting['pred_mean']

In [124]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'Probabilidad': y_pred})

In [125]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']})

## Punto de corte

In [126]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=123
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado


In [127]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)

model_name = 'dataset_10_meses_l'
pred_model_sorted = prediccion.sort_values('Probabilidad', ascending=False)

# Iteramos sobre cada corte
for corte in cortes:
    ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte)
    
    # Almacenamos los resultados
    resultados.append({
        'Modelo': model_name,
        'Corte': corte,
        'Ganancia Público': ganancia_publico,
        'Ganancia Privado': ganancia_privado
    })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 1.80 segundos


In [128]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_dataset_10_meses_l,Ganancia Privado_dataset_10_meses_l
0,5000,-3.196667e+06,-3020000.0
1,6000,-3.686667e+06,-3680000.0
2,7000,-4.410000e+06,-3990000.0
3,8000,-4.946667e+06,-4780000.0
4,9000,-3.850000e+06,-5380000.0
5,10000,-4.573333e+06,-6110000.0
6,11000,-5.366667e+06,-6380000.0
7,12000,-5.810000e+06,-7010000.0
8,13000,-5.530000e+06,-7690000.0
9,14000,-6.276667e+06,-8400000.0


In [101]:
pd.set_option('display.max_rows', 150)
resultados_pivot

,Corte,Ganancia Público_dataset_10_meses_l,Ganancia Privado_dataset_10_meses_l
0,5000,-4.200000e+05,-2960000.0
1,6000,-1.143333e+06,-3790000.0
2,7000,-1.750000e+06,-4480000.0
3,8000,-2.566667e+06,-5200000.0
4,9000,-3.126667e+06,-5810000.0
5,10000,-3.873333e+06,-6610000.0
6,11000,-4.410000e+06,-6930000.0
7,12000,-4.853333e+06,-7560000.0
8,13000,-4.690000e+06,-7720000.0
9,14000,-4.620000e+06,-7940000.0
